In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J.
Editora: CENGAGE LEARNING

Cap. 8: Heterocedasticidade (Heteroskedasticity)
Exemplo 8.8: Participação de mulheres casadas na força de trabalho
             (LABOR FORCE PARTICIPATION OF MARRIED WOMEN)

Arquivo com os dados: mroz.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
df = pd.read_excel('mroz.xls',
                   header=None,
                   usecols=[0, 2, 3, 4, 5, 18, 19, 21],
                   names=['inlf', 'kidslt6', 'kidsge6', 'age', 'educ', 'exper', 'nwifeinc', 'expersq'])

In [3]:
df.head()

,inlf,kidslt6,kidsge6,age,educ,exper,nwifeinc,expersq
0,1,1,0,32,12,14,10.910060,196
1,1,0,2,30,12,5,19.499980,25
2,1,1,3,35,12,15,12.039910,225
3,1,0,3,34,12,6,6.799996,36
4,1,1,2,31,14,7,20.100060,49


### Solução sugerida

In [ ]:
'''
Procedimento para estimar o Modelo de Probabilidade Linear por MQGF (MQGF pois usamos h_chapeu)
Segue o item do livro: Estimando o modelo de probabilidade linear por mínimos quadrados ponderados
'''

In [4]:
# Passo 1: Estime o modelo por MQO e obtenha os valores estimados y_chapeu
modelo_passo_1 = smf.ols('inlf ~ nwifeinc + educ + exper + expersq + age + kidslt6 + kidsge6', data=df)
reg_passo_1 = modelo_passo_1.fit()
y_chapeu = reg_passo_1.fittedvalues   

In [5]:
'''
Passo 2: Se todos os valores estimados (valores em y_chapeu) estiverem no intervalo aberto ]0, 1[, prossiga para o Passo 3.
         Se houver algum valor de y_chapeu fora do intervalo aberto ]0, 1[, abandone MQGF e estime o modelo por MQO
         utilizando as estatísticas robustas em relação à heterocedasticidade.
'''

teste = y_chapeu.between(0, 1, inclusive='neither')
if teste.sum() == teste.size:
    print('Todos os valores estimados estão no intervalo aberto ]0, 1[.\nProssiga para o passo 3')
else:
    print('Há valores em y_chapeu fora do intervalo aberto ]0, 1[.\
    \nEstime o modelo por MQO com as estatísticas robustas em relação à heterocedasticidade.')

Há valores em y_chapeu fora do intervalo aberto ]0, 1[.    
Estime o modelo por MQO com as estatísticas robustas em relação à heterocedasticidade.


In [ ]:
'''
Passo 3: Construa as variâncias estimadas na equação (8.47): 
         h_chapeu = y_chapeu * (1 - y_chapeu)

Não implementado pois no Passo 2 verificou-se que há valores em y_chapeu fora do intervalo aberto ]0, 1[
Para implementação, ver exemplo 8.9.
'''

In [ ]:
'''
Passo 4: Estime a equação do passo 1 por MQP usando peso = 1 / h_chapeu
Não implementado pois no Passo 2 verificou-se que há valores em y_chapeu fora do intervalo aberto ]0, 1[
Para implementação, ver exemplo 8.9.
'''

In [ ]:
# Pela conclusão do Passo 2, vamos estimar o modelo por MQO com os procedimentos robustos em relação à heterocedasticidade.
# Os erros padrão não robustos são colocados na tabela para comparação, seguindo o exemplo do livro.

In [6]:
modelo = smf.ols('inlf ~ nwifeinc + educ + exper + expersq + age + kidslt6 + kidsge6', data=df)
reg = modelo.fit()

In [7]:
tabela = pd.DataFrame({'estimativas MQO': round(reg.params, 3), 
                       'nonrobust SE': round(reg.bse, 3), 
                       'robust SE': round(reg.HC0_se, 3)})
print('Exemplo 8.8 [Variável dependente: inlf]\n')
print(tabela)
print(f'\nObservações: {int(reg.nobs)}')
print(f'R-quadrado: {round(reg.rsquared, 3)}')

Exemplo 8.8 [Variável dependente: inlf]

           estimativas MQO  nonrobust SE  robust SE
Intercept            0.586         0.154      0.151
nwifeinc            -0.003         0.001      0.002
educ                 0.038         0.007      0.007
exper                0.039         0.006      0.006
expersq             -0.001         0.000      0.000
age                 -0.016         0.002      0.002
kidslt6             -0.262         0.034      0.032
kidsge6              0.013         0.013      0.013

Observações: 753
R-quadrado: 0.264


### Comandos comentados (Relacionados ao Passo 2)

In [10]:
# Cria uma série do pandas (apenas para ilustração)
serie = pd.Series([-0.1, 0, 0.5, 1, 1.1])
serie

0   -0.1
1    0.0
2    0.5
3    1.0
4    1.1
dtype: float64

In [9]:
# Abaixo, o método "between()" verifica se os valores da variável "serie" estão no intervalo entre 0 e 1.
# inclusive = 'neither' determina que o intervalo entre 0 e 1 é um intervalo aberto (não inclui os extremos 0 e 1)
# Note que, dos valores constantes na variável "serie", apenas 0.5 está no intervalo aberto ]0, 1[.

teste = serie.between(0, 1, inclusive='neither')
teste

0    False
1    False
2     True
3    False
4    False
dtype: bool

In [11]:
# Em uma Série do pandas com valores booleanos, o método "sum()"" faz a soma o número de valores True
teste.sum() 

1

In [12]:
# O atributo size retorna o número de elementos
teste.size

5

In [13]:
# verifica se o número de elementos True é igual ao número de elementos em "teste" (se há algum False na variável "teste")
# Se todos os elementos da variável "teste" for True, devemos ter True como resultado do teste abaixo.
teste.sum() == teste.size

False